thank for watching and upvote plz

In [ ]:
import optuna
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA

from category_encoders import WOEEncoder

pd.set_option("display.max_columns", None)

train = pd.read_csv('/kaggle/input/playground-series-s3e3/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s3e3/test.csv', index_col='id')
submission = pd.read_csv('/kaggle/input/playground-series-s3e3/sample_submission.csv')
original = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
train['original'] = 0
test['original'] = 0
original['original'] = 1

# Remove outliers
train = train.drop(527).drop(1398).reset_index(drop=True)

target = 'Attrition'
features = train.drop(target, axis=1).columns.to_list()

# Format original
original = original.reset_index().rename(columns={'index':'id'}).set_index('id').drop(['EmployeeNumber'], axis=1)
original['Attrition'] = original['Attrition'].apply(lambda x: 1 if x =='Yes' else 0)

# Concatinate train and original dataset
train = pd.concat([train, original],axis=0).reset_index(drop=True)

float_features = [f for f in features if train[f].dtype == 'float64']
int_features = [f for f in features if train[f].dtype == 'int64']
s = (train.dtypes == 'object')
categorical_features = list(s[s].index)

# drop columns with only one value
for f in features:
    if len(train[f].unique()) == 1 and f != 'original':
        print('Drop', f)
        for ff in [features, float_features, int_features, categorical_features]:
            if f in ff:
                ff.remove(f)
        for df in [train, test]:
            df.drop(f, axis=1, inplace=True) 


# Add features
def young_and_low_daily_rate(x):
    if x['Age'] <= 25 & x['DailyRate'] < 500:
        return 1
    else:
        return 0
    
def overtime_satisfaction(x):
        if x['OverTime'] == 'Yes':
            return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 0.05) * x['JobSatisfaction'])/x['Age']
        else:
            return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 1.05) * x['JobSatisfaction'])/x['Age']
        
for df in [train, test, original]:    
    df['is_young'] = df['Age'].le(25).astype(int)
    df['young_and_underpaid'] = df.apply(lambda x: young_and_low_daily_rate(x), axis = 1)
    df['worklife_stock'] = df.apply(lambda x: x['WorkLifeBalance'] + x['StockOptionLevel'], axis = 1)

    df['income_satisfaction'] = df.apply(lambda x: x['JobSatisfaction'] * x['MonthlyIncome'], axis = 1)
    df['income_level_environ_job_sat'] = df.apply(lambda x: x['EnvironmentSatisfaction']*x['JobSatisfaction'] * (x['MonthlyIncome']/x['JobLevel']), axis = 1)
    df['overtime_stock'] = df.apply(lambda x: overtime_satisfaction(x), axis = 1)
    
features = train.drop(target, axis=1).columns.to_list()

# Encode categorical
categorical_features = ['BusinessTravel', 'Department','Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
               'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus','NumCompaniesWorked', 'OverTime', 
               'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 
                'WorkLifeBalance', 'YearsAtCompany']
y = train[target]            
encoder = WOEEncoder(cols=categorical_features, random_state=42, sigma=0.05, randomized=True, verbose=1)
train = encoder.fit_transform(train[features], train[target])
train[target] = y
test = encoder.transform(test)

test = test.reset_index(drop=True)

In [ ]:
test

In [ ]:
train

In [ ]:
y = train['Attrition']
train=train.drop('Attrition',axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train[train.columns] = scaler.fit_transform(train[train.columns])
test[test.columns] = scaler.transform(test[test.columns])


In [ ]:
X=train
X_test=test

In [ ]:
X_test

In [ ]:
import catboost
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score

n_folds = 5
repeats = 10

MAX_ITER = 15000
PATIENCE = 1000
DISPLAY_FREQ = 100

modelsCB = []
predsCB = []


k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

MODEL_PARAMS = {
                'random_seed': 42,             
                'iterations': MAX_ITER,
                'early_stopping_rounds': PATIENCE,
                'use_best_model': True,
                'eval_metric': 'RMSE',
                'verbose': 1000,
    
                 'depth': 3,
                 'learning_rate': 0.01,
                 'rsm': 0.5,
                 'subsample': 0.931,
                 'l2_leaf_reg': 69,
                 'min_data_in_leaf': 20, 
                 'random_strength': 0.175,

               }
for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    

    model = catboost.CatBoostRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds = PATIENCE,
         )
    modelsCB.append(model)
    predsCB.append(model.predict(X_test))
 

In [ ]:
MAX_ITER = 15000
PATIENCE = 1000
DISPLAY_FREQ = 100

modelsCBC = []
predsCBC = []


k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

MODEL_PARAMS = {
                'random_seed': 42,             
                'iterations': MAX_ITER,
                'early_stopping_rounds': PATIENCE,
                'use_best_model': True,
                'eval_metric': 'AUC',

                'verbose': 1000,
    
                 'depth': 3,
                 'learning_rate': 0.01, 
                 'rsm': 0.5,
                 'subsample': 0.931,
                 'l2_leaf_reg': 69, #69, 4.445
                 'min_data_in_leaf': 20, #20, 1
                 'random_strength': 0.175,
               }


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = catboost.CatBoostClassifier(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          early_stopping_rounds = PATIENCE,
         )
    modelsCBC.append(model)
    predsCBC.append(model.predict_proba(X_test)[:, 1])

In [ ]:
from xgboost import XGBClassifier, XGBRegressor

k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

modelsXB = []
predsXB = []

PATIENCE = 200

MODEL_PARAMS = {       'n_estimators': 1000, 
                       'learning_rate': 0.01, #0.04625397031701272, 0.06733333333333334
                       'max_depth': 9, #3, 29
                       'colsample_bytree': 0.9, #0.9, 0.99
                       'subsample': 1, #1, 0.99
                       'reg_lambda': 20,
#                        'eval_metric': 'auc',
                       'eval_metric': 'rmse',
                       'early_stopping_rounds': PATIENCE,
#                        'tree_method': 'gpu_hist',
                       'seed': 1
}

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = XGBRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          verbose = 100
         )
    modelsXB.append(model)
    predsXB.append(model.predict(X_test))

In [ ]:
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

modelsXBC = []
predsXBC = []

PATIENCE = 200

MODEL_PARAMS = {       'n_estimators': 2000, 
                       'learning_rate': 0.01,
                       'max_depth': 9, #4, #3, 29
                       'colsample_bytree': 0.9,
                       'subsample': 1, #1, 0.99
                       'reg_lambda': 20,
                       'eval_metric': 'auc',
                       'early_stopping_rounds': PATIENCE,
                       'seed': 1
}

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = XGBClassifier(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          verbose = 100
         )
    modelsXBC.append(model)
    predsXBC.append(model.predict_proba(X_test)[:, 1])

In [ ]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier, LGBMRegressor

k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=repeats, random_state=42) 

modelsLB = []
predsLB = []

MODEL_PARAMS = {
                       'learning_rate': 0.01,
                       'max_depth': 9,
                       'num_leaves': 90,
                       'colsample_bytree': 0.8,
                       'subsample': 0.9,
                       'subsample_freq': 5,
                       'min_child_samples': 36,
                       'reg_lambda': 28,
                       'n_estimators': 20000,
                       'metric': 'rmse',
                       'random_state': 42
}

callbacks = [
             lgbm.early_stopping(30, verbose=1),
            ]

for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = lgbm.LGBMRegressor(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
          eval_set=[(X_valid, y_valid)],
          callbacks=callbacks
         )
    modelsLB.append(model)
    predsLB.append(model.predict(X_test))

In [ ]:
from sklearn.linear_model import LassoCV

# n_folds = 20
k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=2*repeats, random_state=42) # 20

modelsLR = []
predsLR = []

MODEL_PARAMS = {
                       'precompute': "auto",
                       'fit_intercept': True,
                       'normalize': False,
                       'max_iter': 1000,
                       'verbose': False,
                       'eps': 1e-04,
                       'cv': 5,
                       'n_alphas': 1000,
                       'n_jobs': 8,
}


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = LassoCV(**MODEL_PARAMS)
    
    model.fit(X=X_train, y=y_train,
         )
    
    modelsLR.append(model)
    predsLR.append(model.predict(X_test))

In [ ]:
from sklearn.linear_model import RidgeCV

k_fold = RepeatedStratifiedKFold(n_splits=n_folds, n_repeats=2*repeats, random_state=42) # 20

modelsR = []
predsR = []


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    model = RidgeCV(alphas=np.linspace(0.0001, 100, 1000)
                   )
    
    model.fit(X=X_train, y=y_train)
    
    modelsR.append(model)
    predsR.append(model.predict(X_test))

In [ ]:
keras_nn_preds = pd.read_csv('/kaggle/input/keras-nn-submission/submission nn.csv')

In [ ]:
predNN = keras_nn_preds.Attrition

In [ ]:
from sklearn.metrics import roc_auc_score

 
def coef_objective(trial):
    a = trial.suggest_float('a', 0, 1)
    b = trial.suggest_float('b', 0, 1)
    c = trial.suggest_float('c', 0, 1)
    d = trial.suggest_float('d', 0, 1)
    e = trial.suggest_float('e', 0, 1)
    f = trial.suggest_float('f', 0, 1)
    g = trial.suggest_float('g', 0, 1)
    h = trial.suggest_float('h', 0, 1)
    
#     X = X1
#     y = y1
    
    preds_eval = []
    for model in modelsCB:
        preds_eval.append(model.predict(X))
    
    resCB = np.average(np.array(preds_eval),axis=0)

    preds_eval = []
    for model in modelsXB:
        preds_eval.append(model.predict(X))
    
    resXB = np.average(np.array(preds_eval),axis=0)

    preds_eval = []
    for model in modelsLB:
        preds_eval.append(model.predict(X))
    
    resLB = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsLR:
        preds_eval.append(model.predict(X))
    
    resLR = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsCBC:
        preds_eval.append(model.predict_proba(X)[:, 1])
    
    resCBC = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsXBC:
        preds_eval.append(model.predict_proba(X)[:, 1])
    
    resXBC = np.average(np.array(preds_eval),axis=0)
    
    preds_eval = []
    for model in modelsR:
        preds_eval.append(model.predict(X))
    
    resR = np.average(np.array(preds_eval),axis=0)
    
    k = pd.read_csv('/kaggle/input/keras-nn-test/KerasNNX1.csv')
    resNN = np.array(k['0'])

    res = roc_auc_score(y, (resCB * a + resXB * b + resLB * c + resLR * d + resNN * e + resCBC * f + resXBC * g + resR * h)/(a + b + c + d + e + f + g + h))

    return res

study = optuna.create_study(direction= 'maximize')

In [ ]:
a = 0.03065532725826637
b = 0.011578190971283535
c = 0.019159393835472383
d = 0.3373901994395983
e = 0.03422756360477424
f = 0.21937795907658797
g = 0.00035405910179546316
h = 0.3472573067122217

In [ ]:
predCB = np.average(np.array(predsCB),axis=0)
predXB = np.average(np.array(predsXB),axis=0)
predLB = np.average(np.array(predsLB),axis=0)
predLR = np.average(np.array(predsLR),axis=0)
predCBC = np.average(np.array(predsCBC),axis=0)
predXBC = np.average(np.array(predsXBC),axis=0)
predR = np.average(np.array(predsR),axis=0)

pred = predCB * a + predXB * b + predLB * c + predLR * d  + predCBC * f + predXBC * g + predR * h  + predNN * e

pred

In [ ]:
submission['Attrition'] = pred
submission
submission.to_csv('submission.csv', index=False)